In [1]:
import time

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch.nn.modules.loss import L1Loss

from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from sklearn import metrics


In [2]:
from google.colab import drive
drive.mount('/content/drive')

np.random.seed(0)

scaler = StandardScaler()

Mounted at /content/drive


In [3]:
# Read Dataset
df = pd.read_csv('/content/drive/MyDrive/ColabNotebooks/4105a6/Housing.csv')

def binary_map(x):
    return x.map({'yes': 1, 'no': 0})

def trinary_map(x):
    return x.map({'unfurnished': 0, 'semi-furnished': 0.5, 'furnished' : 1})

binarymap =  ['mainroad', 'guestroom', 'basement', 'hotwaterheating', 'airconditioning', 'prefarea']
trinarymap = ['furnishingstatus']

# Standardize and turn data into numbers
df[binarymap] = df[binarymap].apply(binary_map)
df[trinarymap] = df[trinarymap].apply(trinary_map)

features = ['area', 'bedrooms', 'bathrooms', 'stories', 'mainroad', 'guestroom', 'basement', 'hotwaterheating', 'airconditioning', 'parking', 'prefarea']
dfX = df[features].values
dfT = df['price'].values
dfX = scaler.fit_transform(dfX)

#split into test/train
Xtrain, Xval, Ttrain, Tval = train_test_split(dfX, dfT, train_size = 0.8, test_size=0.2, random_state=100)

In [4]:
# single hidden layer simple NN class i found online to make code simpler
class SimpleNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

XtrainT = torch.tensor(Xtrain, dtype=torch.float32)
TtrainT = torch.tensor(Ttrain, dtype=torch.float32)
XvalT = torch.tensor(Xval, dtype=torch.float32)
TvalT = torch.tensor(Tval, dtype=torch.float32)

num_epochs = 5000

model = SimpleNN(Xtrain.shape[1], 32, 1)
optimizer = optim.Adam(model.parameters(), lr=0.01)
lossFn = nn.MSELoss()

for epoch in range(num_epochs):
    vpredictions = model(XvalT)
    vloss = lossFn(vpredictions, TvalT.view(-1, 1))

    predictions = model(XtrainT)
    loss = lossFn(predictions, TtrainT.view(-1, 1))

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 500 == 0:
      print('Epoch %d, Loss %f, Validation Loss %f' % (epoch, float(loss), float(vloss)))

nn_complexity = sum(p.numel() for p in model.parameters())
print(f'Neural Network Parameter Count: {nn_complexity}')

Epoch 499, Loss 26419082035200.000000, Validation Loss 25097358278656.000000
Epoch 999, Loss 26239744081920.000000, Validation Loss 24907415027712.000000
Epoch 1499, Loss 25963224104960.000000, Validation Loss 24614799409152.000000
Epoch 1999, Loss 25606429343744.000000, Validation Loss 24237641302016.000000
Epoch 2499, Loss 25181865115648.000000, Validation Loss 23789402324992.000000
Epoch 2999, Loss 24699096530944.000000, Validation Loss 23280499032064.000000
Epoch 3499, Loss 24165721571328.000000, Validation Loss 22719242436608.000000
Epoch 3999, Loss 23587970875392.000000, Validation Loss 22112511197184.000000
Epoch 4499, Loss 22971190083584.000000, Validation Loss 21466238156800.000000
Epoch 4999, Loss 22320122953728.000000, Validation Loss 20785733304320.000000
Neural Network Parameter Count: 417


In [5]:
# multi hidden layer simple NN class i found online to make code simpler

class MultiLayerNN(nn.Module):
    def __init__(self, input_size, hidden_sizes, output_size):
        super(MultiLayerNN, self).__init__()
        self.hidden_layers = nn.ModuleList()
        input_dim = input_size

        for hidden_size in hidden_sizes:
            self.hidden_layers.append(nn.Linear(input_dim, hidden_size))
            self.hidden_layers.append(nn.ReLU())
            input_dim = hidden_size

        self.output_layer = nn.Linear(hidden_sizes[-1], output_size)

    def forward(self, x):
        for layer in self.hidden_layers:
            x = layer(x)
        x = self.output_layer(x)
        return x

XtrainT = torch.tensor(Xtrain, dtype=torch.float32)
TtrainT = torch.tensor(Ttrain, dtype=torch.float32)
XvalT = torch.tensor(Xval, dtype=torch.float32)
TvalT = torch.tensor(Tval, dtype=torch.float32)

num_epochs = 5000

model = MultiLayerNN(Xtrain.shape[1], [32, 64, 16], 1)
optimizer = optim.Adam(model.parameters(), lr=0.01)
lossFn = nn.MSELoss()

for epoch in range(num_epochs):
    vpredictions = model(XvalT)
    vloss = lossFn(vpredictions, TvalT.view(-1, 1))

    predictions = model(XtrainT)
    loss = lossFn(predictions, TtrainT.view(-1, 1))

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 500 == 0:
      print('Epoch %d, Loss %f, Validation Loss %f' % (epoch, float(loss), float(vloss)))

nn_complexity = sum(p.numel() for p in model.parameters())
print(f'Neural Network Parameter Count: {nn_complexity}')

Epoch 499, Loss 1127438483456.000000, Validation Loss 1302031761408.000000
Epoch 999, Loss 977011867648.000000, Validation Loss 1193546350592.000000
Epoch 1499, Loss 915788398592.000000, Validation Loss 1169670144000.000000
Epoch 1999, Loss 884301824000.000000, Validation Loss 1153135542272.000000
Epoch 2499, Loss 863492702208.000000, Validation Loss 1147552661504.000000
Epoch 2999, Loss 841522544640.000000, Validation Loss 1150535729152.000000
Epoch 3499, Loss 826477903872.000000, Validation Loss 1177332875264.000000
Epoch 3999, Loss 813984317440.000000, Validation Loss 1179661369344.000000
Epoch 4499, Loss 802198650880.000000, Validation Loss 1201714757632.000000
Epoch 4999, Loss 712676147200.000000, Validation Loss 1334610755584.000000
Neural Network Parameter Count: 3553


In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

#same class as above, needed a flatten line to function here though
class SimpleNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=False, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=2)

#Defs
model = SimpleNN(32 * 32 * 3 , 512, 10).to(device)
lossfn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
num_epochs = 300
start_time = time.time()

#Training Loop
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = lossfn(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    epoch_loss = running_loss / len(train_loader)
    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {epoch_loss:.4f}")

training_time = time.time() - start_time


cuda


100%|██████████| 170498071/170498071 [00:06<00:00, 26266920.34it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified
Epoch 1/300, Loss: 1.6581
Epoch 2/300, Loss: 1.4640
Epoch 3/300, Loss: 1.3681
Epoch 4/300, Loss: 1.2877
Epoch 5/300, Loss: 1.2163
Epoch 6/300, Loss: 1.1458
Epoch 7/300, Loss: 1.0780
Epoch 8/300, Loss: 1.0217
Epoch 9/300, Loss: 0.9607
Epoch 10/300, Loss: 0.9110
Epoch 11/300, Loss: 0.8672
Epoch 12/300, Loss: 0.8302
Epoch 13/300, Loss: 0.8014
Epoch 14/300, Loss: 0.7567
Epoch 15/300, Loss: 0.7322
Epoch 16/300, Loss: 0.7164
Epoch 17/300, Loss: 0.6738
Epoch 18/300, Loss: 0.6545
Epoch 19/300, Loss: 0.6351
Epoch 20/300, Loss: 0.6132
Epoch 21/300, Loss: 0.6161
Epoch 22/300, Loss: 0.5931
Epoch 23/300, Loss: 0.5662
Epoch 24/300, Loss: 0.5585
Epoch 25/300, Loss: 0.5495
Epoch 26/300, Loss: 0.5358
Epoch 27/300, Loss: 0.5217
Epoch 28/300, Loss: 0.4975
Epoch 29/300, Loss: 0.4829
Epoch 30/300, Loss: 0.4614
Epoch 31/300, Loss: 0.4597
Epoch 32/300, Loss: 0.4534
Epoch 33/300, Loss: 0.4402
Epoch 34/300, Loss: 0.4309
Ep

In [7]:
#2a diagnostics
print(f"Training Time: {training_time:.2f} seconds")

model.eval()
all_predictions = []
true_labels = []
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        outputs = model(inputs)
        _, predictions = torch.max(outputs, 1)

        all_predictions.extend(predictions.cpu().numpy())
        true_labels.extend(labels.cpu().numpy())
accuracy = accuracy_score(true_labels, all_predictions)
f1 = f1_score(true_labels, all_predictions, average='weighted')
print(f"Evaluation Accuracy: {accuracy * 100:.2f}%")
print(f"F1 Score: {f1 * 100:.2f}%")
nn_complexity = sum(p.numel() for p in model.parameters())
print(f'Neural Network Parameter Count: {nn_complexity}')

conf_matrix = confusion_matrix(true_labels, all_predictions)
print("Confusion Matrix:")
print(conf_matrix)

Training Time: 2267.06 seconds
Evaluation Accuracy: 50.81%
F1 Score: 50.92%
Neural Network Parameter Count: 1578506
Confusion Matrix:
[[571  54  94  27  23  18  18  12 121  62]
 [ 37 650  36  23   6  17  13  14  71 133]
 [ 80  21 455  73 115  88  66  64  15  23]
 [ 29  37 118 331  60 221  90  58  21  35]
 [ 38  26 170  79 416  77  58  86  22  28]
 [ 20  14 119 184  63 428  48  89  17  18]
 [ 14  35 109 106  91  73 509  20  13  30]
 [ 33  21  85  53  79 110  18 549  13  39]
 [ 96  72  43  37  15  15  14   8 645  55]
 [ 40 183  41  41  19  24  25  26  74 527]]


In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#same class as above, added flattening
class MultiLayerNN(nn.Module):
    def __init__(self, input_size, hidden_sizes, output_size):
        super(MultiLayerNN, self).__init__()
        self.hidden_layers = nn.ModuleList()
        input_dim = input_size

        for hidden_size in hidden_sizes:
            self.hidden_layers.append(nn.Linear(input_dim, hidden_size))
            self.hidden_layers.append(nn.ReLU())
            input_dim = hidden_size

        self.output_layer = nn.Linear(hidden_sizes[-1], output_size)

    def forward(self, x):
        x = x.view(x.size(0), -1)
        for layer in self.hidden_layers:
            x = layer(x)
        x = self.output_layer(x)
        return x

transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=False, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=2)

#Defs
model = MultiLayerNN(32 * 32 * 3, [512,1024,256], 10).to(device)
lossfn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 300
start_time = time.time()

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = lossfn(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    epoch_loss = running_loss / len(train_loader)
    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {epoch_loss:.4f}")

training_time = time.time() - start_time

Files already downloaded and verified
Files already downloaded and verified
Epoch 1/300, Loss: 1.6642
Epoch 2/300, Loss: 1.4317
Epoch 3/300, Loss: 1.2906
Epoch 4/300, Loss: 1.1704
Epoch 5/300, Loss: 1.0579
Epoch 6/300, Loss: 0.9543
Epoch 7/300, Loss: 0.8627
Epoch 8/300, Loss: 0.7815
Epoch 9/300, Loss: 0.7017
Epoch 10/300, Loss: 0.6419
Epoch 11/300, Loss: 0.5987
Epoch 12/300, Loss: 0.5364
Epoch 13/300, Loss: 0.5040
Epoch 14/300, Loss: 0.4771
Epoch 15/300, Loss: 0.4432
Epoch 16/300, Loss: 0.4081
Epoch 17/300, Loss: 0.4040
Epoch 18/300, Loss: 0.3835
Epoch 19/300, Loss: 0.3506
Epoch 20/300, Loss: 0.3419
Epoch 21/300, Loss: 0.3207
Epoch 22/300, Loss: 0.3240
Epoch 23/300, Loss: 0.2985
Epoch 24/300, Loss: 0.3030
Epoch 25/300, Loss: 0.2931
Epoch 26/300, Loss: 0.2707
Epoch 27/300, Loss: 0.2679
Epoch 28/300, Loss: 0.2672
Epoch 29/300, Loss: 0.2585
Epoch 30/300, Loss: 0.2570
Epoch 31/300, Loss: 0.2352
Epoch 32/300, Loss: 0.2429
Epoch 33/300, Loss: 0.2350
Epoch 34/300, Loss: 0.2321
Epoch 35/300, L

In [10]:
#2b diagnostics
print(f"Training Time: {training_time:.2f} seconds")

model.eval()
all_predictions = []
true_labels = []
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        outputs = model(inputs)
        _, predictions = torch.max(outputs, 1)

        all_predictions.extend(predictions.cpu().numpy())
        true_labels.extend(labels.cpu().numpy())
accuracy = accuracy_score(true_labels, all_predictions)
f1 = f1_score(true_labels, all_predictions, average='weighted')
print(f"Evaluation Accuracy: {accuracy * 100:.2f}%")
print(f"F1 Score: {f1 * 100:.2f}%")
nn_complexity = sum(p.numel() for p in model.parameters())
print(f'Neural Network Parameter Count: {nn_complexity}')

conf_matrix = confusion_matrix(true_labels, all_predictions)
print("Confusion Matrix:")
print(conf_matrix)

Training Time: 2301.72 seconds
Evaluation Accuracy: 52.49%
F1 Score: 52.53%
Neural Network Parameter Count: 2363658
Confusion Matrix:
[[617  18  60  21  68  22  21  30  98  45]
 [ 52 573  25  29  37   8  22  22  87 145]
 [ 72   7 396  80 179  61  96  66  22  21]
 [ 27  16  97 369 125 147 102  74  13  30]
 [ 41   6 118  67 464  56 102 104  23  19]
 [ 25  11  85 218  99 364  52 117  11  18]
 [ 12  17  60  95  89  46 638  16  10  17]
 [ 34  10  62  59  98  56  21 617  13  30]
 [115  43  24  35  56  14  13  19 639  42]
 [ 57 109  26  35  36  20  20  53  72 572]]
